In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [3]:
twitter_datadir = 'E:/VU/subjectivity_mining/eigenBert/twitterdata/'
waseem_data = pd.read_csv(f'{twitter_datadir}final.csv', sep=';')


## Waseem data 1 & 2

In [4]:
waseem_data.head()

,tweet,label_true,tweet_id
0,NaN,not_sexism,572342978255048705
1,NaN,not_sexism,572341498827522049
2,NaN,not_sexism,572340476503724032
3,NaN,not_sexism,572334712804384768
4,NaN,not_sexism,572332655397629952


In [5]:
#This code creates the dataset from Corpus.csv which is downloadable from the
#internet well known dataset which is labeled manually by hand. But for the text
#of tweets you need to fetch them with their IDs.
import tweepy
import time

# Twitter Developer keys here
# It is CENSORED
consumer_key = 'lGxf46TK5R04JxJTlq1HG1s0o'
consumer_key_secret = 'ikBv6TQclWcxNS8KTzxqO13i8NsnXyvNU6m1KbI64D290fhXqe'
access_token = '1446788827599736833-6RP9ATN2dAb9XXMz2xCKl0CjSXAu5y'
access_token_secret = 'uk8BgEi5RICOUwUC06V4S5QfQ5REjvugF7sTNwXYGwb8N'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# time.sleep(900) # Delay for 15 minutes
# print('sleeping 15 min..')

# This method creates the training set
def createTrainingSet(corpusFile, targetResultFile):
    import csv
    import time

    count = 0
    counter = 0
    corpus = []

    with open(corpusFile, 'r') as csvfile:
        lineReader = csv.reader(csvfile, delimiter=';', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id": row[2], "label": row[1], "topic": row[0]})

    sleepTime = 2
    trainingDataSet = []

    for tweet in corpus:
        try:
            tweetFetched = api.get_status(tweet["tweet_id"])
            print(tweet['tweet_id'])
            print("Tweet fetched" + tweetFetched.text)
            tweet["text"] = tweetFetched.text
            trainingDataSet.append(tweet)
            time.sleep(sleepTime)

        except tweepy.TweepyException as e:
            print('the error code:', print(e))
            # print('the error message:', tweepy.error.TweepError.message[0][0]['message'])  

        # except:
        #     print("Inside the exception - no:2")
        #     print(tweet['tweet_id'])
        #     print ('the error code:', e.args[0][0]['code'])
        #     print ('the error message:', e.args[0][0]['message']) 
        #     continue

        count += 1
        if count % 900 == 0:
            print('sleeping..')
            time.sleep(900) # Delay for 15 minutes

    with open(targetResultFile, 'w') as csvfile:
        linewriter = csv.writer(csvfile, delimiter=',', quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)


    return trainingDataSet

# Code starts here
# This is corpus dataset
corpusFile = 'E:/VU/subjectivity_mining/eigenBert/twitterdata/pure_sexist.csv'
# This is my target file
targetResultFile = "E:/VU/subjectivity_mining/eigenBert/twitterdata/pure_sexist_filled.csv"
# Call the method
resultFile = createTrainingSet(corpusFile, targetResultFile)

404 Not Found
8 - No data available for specified ID.
the error code: None
572346080911736832
Tweet fetchedThese two are revolting #MKR #MKR2015
572348198062170112
Tweet fetched#katieandnikki stop calling yourselves pretty and hot..you're not and saying it a million times doesn't make you either...STFU #MKR
404 Not Found
144 - No status found with that ID.
the error code: None
572319306387599360
Tweet fetchedThe menus look like they were made by a 5 year old little girl...in this case just the mental age of a 5 year old girl I guess #MKR
572347842456522752
Tweet fetchedWish these blondes were in that How To Get Away With Murder show....#MKR
572347584339046402
Tweet fetchedWere butchers but can't tell red poultry is not cooked..  #salmonellaqueens #MKR
572347419209252865
Tweet fetchedNot dumb blondes...mmm...perhaps just delusions of adequacy? #MKR
572347385889759232
Tweet fetchedStop saying dumb blondes with pretty faces as you need a pretty face to pull that off!!!! #mkr
5723217338410